# Clustering Neighborhoods in Toronto

In [38]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [39]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [40]:
neighborhoods = data_comb[['Borough', 'Neighborhood','Latitude','Longitude']]
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [41]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto City.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [42]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [43]:
# create map of Torento using latitude and longitude values
map_torento = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_torento)  
    
map_torento

# Define Foursquare Credentials and Version

In [44]:
CLIENT_ID = '5I4CX2DSHRKH42AL3ZQCBCYF2JPFLN3WCW5Z1S2HRBDGDJWW' # your Foursquare ID
CLIENT_SECRET = '3VDDIH24LT4YKEU3OM3IMKPA3O5EKLHZHCGIX0QPPWY2KOLE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5I4CX2DSHRKH42AL3ZQCBCYF2JPFLN3WCW5Z1S2HRBDGDJWW
CLIENT_SECRET:3VDDIH24LT4YKEU3OM3IMKPA3O5EKLHZHCGIX0QPPWY2KOLE


In [45]:
northyork_data = neighborhoods[neighborhoods['Borough'] == 'North York'].reset_index(drop=True)
northyork_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Hillcrest Village,43.803762,-79.363452
1,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556
2,North York,Bayview Village,43.786947,-79.385975
3,North York,"Silver Hills,York Mills",43.757490,-79.374714
4,North York,"Newtonbrook,Willowdale",43.789053,-79.408493


In [46]:
address = 'North York'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7708175, -79.4132998.


In [47]:
# create map of Manhattan using latitude and longitude values
map_northyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)  
    
map_northyork

# Let's explore the first neighborhood in our dataframe.
Get the neighborhood's name.

In [48]:
northyork_data.loc[0, 'Neighborhood']

'Hillcrest Village'

Get the neighborhood's latitude and longitude values.

In [49]:
neighborhood_latitude = northyork_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = northyork_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = northyork_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Hillcrest Village are 43.8037622, -79.3634517.


## Now, let's get the top 100 venues that are in Hillcrest Village within a radius of 500 meters.
First, let's create the GET request URL. Name your URL url.

In [50]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=5I4CX2DSHRKH42AL3ZQCBCYF2JPFLN3WCW5Z1S2HRBDGDJWW&client_secret=3VDDIH24LT4YKEU3OM3IMKPA3O5EKLHZHCGIX0QPPWY2KOLE&v=20180605&ll=43.8037622,-79.3634517&radius=500&limit=100'

In [51]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ce3de95db04f52f5e877791'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4ad9dce6f964a520651b21e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/golfcourse_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1e6941735',
         'name': 'Golf Course',
         'pluralName': 'Golf Courses',
         'primary': True,
         'shortName': 'Golf Course'}],
       'id': '4ad9dce6f964a520651b21e3',
       'location': {'address': '10000 Dufferin Rd',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 197,
        'formattedAddress': ['10000 Dufferin Rd', 'Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.805454826002794,
    

In [52]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [53]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Eagle's Nest Golf Club,Golf Course,43.805455,-79.364186
1,AY Jackson Pool,Pool,43.804515,-79.366138
2,Villa Madina,Mediterranean Restaurant,43.801685,-79.363938
3,Duncan Creek Park,Dog Run,43.805539,-79.360695


In [54]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


# Explore Neighborhoods in North York
## Let's create a function to repeat the same process to all the neighborhoods in North York

In [55]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [56]:
northyork_venues = getNearbyVenues(names=northyork_data['Neighborhood'],
                                   latitudes=northyork_data['Latitude'],
                                   longitudes=northyork_data['Longitude']
                                  )

Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park,Lawrence Manor East
Lawrence Heights,Lawrence Manor
Glencairn
Downsview,North Park,Upwood Park
Humber Summit
Emery,Humberlea


In [57]:
print(northyork_venues.shape)
northyork_venues.head()

(243, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store


Let's check how many venues were returned for each neighborhood

In [58]:
northyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",22,22,22,22,22,22
"CFB Toronto,Downsview East",3,3,3,3,3,3
Don Mills North,6,6,6,6,6,6
Downsview Central,2,2,2,2,2,2
Downsview Northwest,4,4,4,4,4,4
Downsview West,5,5,5,5,5,5
"Downsview,North Park,Upwood Park",4,4,4,4,4,4


## Let's find out how many unique categories can be curated from all the returned venues

In [59]:
print('There are {} uniques categories.'.format(len(northyork_venues['Venue Category'].unique())))

There are 109 uniques categories.


# 3. Analyze Each Neighborhood

In [60]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

for hood in northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor,Downsview North,Wilson Heights----
                venue  freq
0         Coffee Shop  0.11
1  Frozen Yogurt Shop  0.06
2         Bridal Shop  0.06
3         Pizza Place  0.06
4       Deli / Bodega  0.06


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2  Japanese Restaurant  0.25
3                 Café  0.25
4    Accessories Store  0.00


----Bedford Park,Lawrence Manor East----
                  venue  freq
0  Fast Food Restaurant  0.09
1    Italian Restaurant  0.09
2           Coffee Shop  0.09
3      Sushi Restaurant  0.05
4                  Café  0.05


----CFB Toronto,Downsview East----
         venue  freq
0      Airport  0.33
1         Park  0.33
2  Snack Place  0.33
3        Plaza  0.00
4     Pharmacy  0.00


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center  0.17
1   Japanese Restaurant  0.17
2  Caribbean Restaurant  0.17
3        Baseball Field  0.17
4      Basketbal

### Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [61]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [62]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Fast Food Restaurant,Middle Eastern Restaurant,Pizza Place,Deli / Bodega,Diner,Restaurant,Sandwich Place,Bridal Shop,Shopping Mall
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Yoga Studio,Empanada Restaurant,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
2,"Bedford Park,Lawrence Manor East",Italian Restaurant,Coffee Shop,Fast Food Restaurant,Indian Restaurant,Pharmacy,Pizza Place,Café,Liquor Store,Butcher,Pub
3,"CFB Toronto,Downsview East",Airport,Park,Snack Place,Yoga Studio,Electronics Store,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
4,Don Mills North,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Basketball Court,Japanese Restaurant,Event Space,Deli / Bodega,Department Store,Dim Sum Restaurant


# 4. Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.

In [63]:
# set number of clusters
kclusters = 5

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 



array([0, 0, 0, 0, 0, 2, 0, 0, 0, 1], dtype=int32)

In [64]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

northyork_merged = northyork_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

northyork_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Hillcrest Village,43.803762,-79.363452,0.0,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Yoga Studio,Eastern European Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
1,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,0.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Gift Shop,Food Court,Restaurant,Bakery,Kids Store,Japanese Restaurant,Shoe Store
2,North York,Bayview Village,43.786947,-79.385975,0.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Yoga Studio,Empanada Restaurant,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
3,North York,"Silver Hills,York Mills",43.757490,-79.374714,4.0,Cafeteria,Yoga Studio,Empanada Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store
4,North York,"Newtonbrook,Willowdale",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 5. Examine Clusters
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

## Cluster 1

In [66]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 0, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hillcrest Village,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Yoga Studio,Eastern European Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
1,"Fairview,Henry Farm,Oriole",Clothing Store,Fast Food Restaurant,Coffee Shop,Gift Shop,Food Court,Restaurant,Bakery,Kids Store,Japanese Restaurant,Shoe Store
2,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Yoga Studio,Empanada Restaurant,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
5,Willowdale South,Coffee Shop,Ramen Restaurant,Sandwich Place,Sushi Restaurant,Café,Japanese Restaurant,Restaurant,Bubble Tea Shop,Indonesian Restaurant,Pizza Place
6,York Mills West,Convenience Store,Park,Bank,Yoga Studio,Empanada Restaurant,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner
7,Willowdale West,Coffee Shop,Pizza Place,Butcher,Discount Store,Pharmacy,Furniture / Home Store,Gift Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
8,Parkwoods,Park,Food & Drink Shop,Fast Food Restaurant,Yoga Studio,Electronics Store,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
9,Don Mills North,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Basketball Court,Japanese Restaurant,Event Space,Deli / Bodega,Department Store,Dim Sum Restaurant
10,"Flemingdon Park,Don Mills South",Grocery Store,Gym,Coffee Shop,Asian Restaurant,Beer Store,Clothing Store,Sporting Goods Shop,Chinese Restaurant,Dim Sum Restaurant,Discount Store
11,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Fast Food Restaurant,Middle Eastern Restaurant,Pizza Place,Deli / Bodega,Diner,Restaurant,Sandwich Place,Bridal Shop,Shopping Mall


## Cluster 2

In [68]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 1, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,"Emery,Humberlea",Baseball Field,Yoga Studio,Event Space,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store


## Cluster 3

In [69]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 3, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Humber Summit,Empanada Restaurant,Pizza Place,Yoga Studio,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner


## Cluster 4

In [70]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 4, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"Silver Hills,York Mills",Cafeteria,Yoga Studio,Empanada Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store


# Cluster 5

In [71]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 5, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
